&emsp;&emsp;在`seq2seq`中，我们使用了贪心搜索来预测序列，也就是将当前时刻预测概率最大的词输出。但是贪心很可能不是最优的。

&emsp;&emsp;**最优算法**：对所有可能的序列，计算它的概率，然后选取最好的那个。如果输出字典大小为$n$，序列最长为$T$，那我们需要考察$n^{T}$个序列。$n = 10000$, $T = 10$, $n^{T} = 10^{40}$，计算上是不可行的。

## 束搜索 (beam search)

&emsp;&emsp;`beam search`是保存最好的$k$个候选，在每个时刻，对每个候选新加一项($n$种可能)，在$kn$个选项中选出最好的$k$个。

具体思路是:

&emsp;&emsp;束搜索(`beam search`)是贪心搜索的一个改进版本。它有一个超参数，名为束宽(`beam size`) $k$。在时间步`1`，我们选择具有最高条件概率的$k$个词元。这个$k$个词元将分别是$k$个候选输出序列的第一个词元。在随后的每个时间步上，基于上一时间步的$k$个候选输出序列，我们将继续从$k |\mathcal(Y)|$个可能的选择中挑选出具有最高条件概率的$k$个候选输出序列。

## 爬山算法

## 代码实现

&emsp;&emsp;最大化如下目标函数:

$$
\begin{aligned}
&f_{1}(x, y)=\sin (2 x)+\cos \left(\frac{y}{2}\right) \\
&f_{2}(x, y)=|x-2|+|0.5 y+1|-4
\end{aligned}
$$

&emsp;&emsp;其中假定$0 \leq x, y \leq 10$。

&emsp;&emsp;定义两个目标函数

In [ ]:
import math

def func1(x, y):
    """
    函数1
    """
    return math.sin(2 * x) + math.cos(y / 2)

def func2(x, y):
    """
    函数2
    """
    return math.fabs(x - 2) + math.fabs(0.5 * y + 1) - 4

### 爬山算法代码实现

In [ ]:
def outofbounds(L):
    """
    # 输入一个坐标元组的List, 挑选出里面没有越界的元组。
    """
    res = []
    for item in L:
        x_1 = item[0]
        y_1 = item[1]
        if(x_1 <= 10 and x_1 >= 0 and y_1 <= 10 and y_1 >= 0):
            res.append(item)
    return res

def nextstep(x, y, stepsize, func):
    """
    # 输入当前的坐标x和y, 以及需要偏移的步长的size，和衡量函数，输出最大值和其对应的坐标。
    """
    neighbours = [(x+stepsize, y),(x, y+stepsize),(x+stepsize, y+stepsize),
                  (x-stepsize, y),(x, y-stepsize),(x-stepsize, y-stepsize),
                  (x-stepsize, y+stepsize),(x+stepsize, y-stepsize),(x, y)]
    
    neighbours = outofbounds(neighbours)  # 剔除掉越界的元素。
    
    # 计算neighbours中每个元素的函数值，并存放在evaluate_list中。
    evaluate_list = [func(neighbour[0], neighbour[1]) for neighbour in neighbours]
    
    maximum = max(evaluate_list)
    
    max_neighbour = neighbours[evaluate_list.index(maximum)]
    return ((maximum, max_neighbour))

In [ ]:
stepsizes = [0.01, 0.05, 0.1, 0.2]
allcounts = []

import numpy as np
nums_cnt = 100
X = np.random.choice(10, nums_cnt)
Y = np.random.choice(10, nums_cnt)
print('X:', X)
print('Y:', Y)

In [ ]:
for j in range(0, len(stepsizes)):
    cnt = 0  # 计数
    difs = []
    res = []
    for i in range(0, nums_cnt):
        x_1 = X[i]
        y_1 = Y[i]
        past_value = []  # 记录最大的值。
        past_neigh = []  # 记录最大的值的坐标。
        
        while True:
            max_value_neigh = nextstep(x_1, y_1, stepsizes[j], func1)
            print(max_value_neigh)
            
            if(cnt > 5):  # 当存入了5个值的时候
                sdifs = difs[len(difs) - 5 : len(difs)]
                if(sum(sdifs) / len(sdifs) < 0.0001):  # 五个差值平均值小于0.0001时，判定为收敛了。
                    print(sdifs)
                    print('Converged!')
                    break
                # 坐标位置不动或者值变化小于0.001的时候，结束。
                if(past_neigh[len(past_neigh) - 2] == max_value_neigh[1] 
                                       or math.fabs(past_value[len(past_value) - 1] - max_value_neigh[0]) < 0.001):
                    print("Done")
                    break
            
            if(cnt > 1):
                # 添加最大值的差值到difs中去。
                difs.append(math.fabs(past_value[len(past_value) - 1] - max_value_neigh[0]))
            
            past_value.append(max_value_neigh[0])
            past_neigh.append(max_value_neigh[1])
            
            cnt += 1
            # 更新x, y坐标
            x_1 = max_value_neigh[1][0]
            y_1 = max_value_neigh[1][1]
        res.append([X[i], Y[i], cnt, past_neigh[0]])
        cnt = 0

### 束搜索代码实现

In [ ]:
stepsizes = [0.01, 0.05, 0.1, 0.2]
widths = [2, 4, 8, 16]
for k in range(0, len(widths)):
    for j in range(0, len(stepsizes)):
        cnt = 0
        res = []
        for i in range(0, 100):
            # 生成k个初始值。
            init_x = np.random.choice(10, widths[k])
            init_y = np.random.choice(10, widths[k])
            
            x_1 = init_x
            y_1 = init_y
            past = []
            while True:
                child = []
                for i in range(0, widths[k]):
                    n = nextstep(x_1[i], y_1[i], stepsizes[j], func2)
                    child.append(n)
                
                # 排序之后取k个。
                sort = sorted(child, key = lambda x:x[0], reverse = True)
                sort = sort[0: widths[k]]
                sort2 = np.array(sort)

                ssum = np.sum(sort2, axis = 0)

                if(cnt > 10):
                    end = True
                    reason = []
                    for i in range(0, len(child)):
                        if(past_children[i] != child[i]):
                            end = False
                    if(end == True):
                        print(sort)
                        print("Converged ALL")
                        
                past_sort = ssum[0] / len(sort)
                past.append(sort)
                past_children = child
                
                cnt += 1
                x_1 = [x[1][0] for x in child]
                y_1 = [y[1][1] for y in child]
            # ['x','y','Counts','Max']
            res.append([x_1[i], y_1[i], cnt, sort[0][0]])
            cnt = 0

In [ ]:
#Beam Search
def beam(x,y,width,func,stepsize):
    init_x = [rand.randrange(0,10) for i in range(0,width)]
    init_y = [rand.randrange(0,10) for i in range(0,width)]
    
    #generate children
    x = init_x
    y = init_y
    
    child=[]
    for i in range(0,width):
        n = nextstep(x[i],y[i],stepsize,func)
        child.append(n)
        
    sort = sorted(child,key=lambda x:x[0])
    sort = sort[0:width]
    return(sort)

## 参考

- [Beam-Search-vs-Hill-Climbing](https://github.com/RScicomp/Beam-Search-vs-Hill-Climbing)